In [19]:
pip list > demo_piplist.txt

Note: you may need to restart the kernel to use updated packages.


In [21]:
!dir

 Volume in drive D is Storage
 Volume Serial Number is E457-CEE6

 Directory of D:\Lemonrice\virtual_demo

09/03/2023  14:48    <DIR>          .
09/03/2023  14:48    <DIR>          ..
09/03/2023  12:01    <DIR>          .ipynb_checkpoints
09/03/2023  13:35               203 1.0
09/03/2023  13:45               203 1.0.0
09/03/2023  13:34               203 1.2
26/03/2014  11:22         5,834,924 bank-additional-full.csv
14/02/2012  14:40         4,610,348 bank-full.csv
09/03/2023  13:01            52,454 Bank_campaign_ML.ipynb
06/03/2023  11:22    <DIR>          bank_full_yr
01/03/2023  23:09            26,955 date_map.ipynb
02/03/2023  14:04            37,059 date_map_colab_32_1.ipynb
02/03/2023  19:50            64,046 date_map_colab_32_2.ipynb
09/03/2023  14:48             4,032 demo_piplist.txt
09/03/2023  13:19           104,890 Mandar_Spark_Bank_Campaign_3_9_1.ipynb
01/03/2023  22:58            32,510 map_date2.ipynb
09/03/2023  12:46           104,078 ml_3_8_1_jupy.ipynb
03/03/202

In [1]:
import findspark
findspark.init()

In [2]:
#importing necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import *

from pyspark.sql.functions import when, col
from pyspark.sql.functions import sum, count
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

# Creating a spark session object

In [3]:
spark = SparkSession.builder.master('local[4]').appName('ProjectLemon').getOrCreate()

# Preprocessing

## Preprocessing on dataset bank-full

In [4]:
# to display dataframe properly in jupyter
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [5]:
bank_full = spark.read.csv('bank-full.csv',
                           sep = ';',
                           header=True,
                           inferSchema=True)

In [6]:
bank_full.count()

45211

In [7]:
bank_full.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

In [8]:
bank_full.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [9]:
new_cols = ["emp_var_rate", "cons_price_idx", "cons_conf_idx", "euribor_3m", "nr_employed"]

for column in new_cols:
    bank_full = bank_full.withColumn(column, bank_full["poutcome"] + 1)

In [10]:
display(bank_full.limit(7))

age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,emp_var_rate,cons_price_idx,cons_conf_idx,euribor_3m,nr_employed
58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,null,null,null,null,null
44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,null,null,null,null,null
33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,null,null,null,null,null
47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,null,null,null,null,null
33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,null,null,null,null,null
35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no,null,null,null,null,null
28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no,null,null,null,null,null


### Dropping the balance column

In [11]:
bank_full.drop("balance").limit(3)

age,job,marital,education,default,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,emp_var_rate,cons_price_idx,cons_conf_idx,euribor_3m,nr_employed
58,management,married,tertiary,no,yes,no,unknown,5,may,261,1,-1,0,unknown,no,null,null,null,null,null
44,technician,single,secondary,no,yes,no,unknown,5,may,151,1,-1,0,unknown,no,null,null,null,null,null
33,entrepreneur,married,secondary,no,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,null,null,null,null,null


In [12]:
display(bank_full.limit(5))

age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,emp_var_rate,cons_price_idx,cons_conf_idx,euribor_3m,nr_employed
58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,null,null,null,null,null
44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,null,null,null,null,null
33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,null,null,null,null,null
47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,null,null,null,null,null
33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,null,null,null,null,null


### Adding year mapper function to add year in DataFrame bank_full

In [13]:
#!pip install pyspark[sql]

In [17]:

#import pyarrow
import pyarrow.lib as _lib
import pandas as pd
from pyspark.sql.functions import col
import pyspark.pandas as ps

import warnings

ImportError: DLL load failed while importing lib: The specified procedure could not be found.

In [ ]:
def year_mapper(data, start_yr, end_yr):
    month_lst = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

    # Make a copy of the original dataframe
    new_data = data.copy()

    # Insert a new "year" column filled with zeros
    new_data.insert(loc=0, column="year", value=0)

    # Set the first year to the start year
    current_year = int(start_yr)
    new_data.at[0, "year"] = current_year

    # Loop through the rows of the dataframe, updating the year column when the month changes
    for i in range(1, len(new_data)):
        # If the current month is earlier in the year than the previous month, increment the year
        if month_lst.index(new_data["month"][i]) < month_lst.index(new_data["month"][i-1]):
            current_year += 1

        new_data.at[i, "year"] = current_year

        # If the current year exceeds the end year, break out of the loop
        if current_year > end_yr:
            break

    return new_data

In [ ]:
# Use default index prevent overhead.
ps.set_option("compute.default_index_type", "distributed") 

warnings.filterwarnings("ignore")  # Ignore warnings coming from Arrow optimizations.

# To speed up dataset processing
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)

bank_full_pdf = bank_full.toPandas()

# Apply the function to the Pandas DataFrame
new_pandas_df = year_mapper(bank_full_pdf, 2008, 2010)

# Convert the updated Pandas DataFrame back to a PySpark DataFrame
bank_full_yr = spark.createDataFrame(new_pandas_df)

bank_full_yr.show()

In [ ]:
result_bankfull = year_mapper( data = , start_yr = 2008)
result_bankfull.head()